In [93]:
import numpy as np
import math 
from svmutil import *
import random
import multiprocessing as mp
from collections import Counter
    

def read_data(dataName):
    yTrain, xTrain = svm_read_problem(dataName)
    return yTrain, xTrain



def to_binary_class(y,label):
    newy = y.copy()
    for i in range(len(y)):
        if newy[i] != label:
            newy[i] = -1
        else:
            newy[i] = 1
    return newy

def sparse_to_dense(xTrain):
    x = xTrain.copy()
    newX = np.zeros(36)
    for index in range(36):
        i = index +1
        if x.get(i) != None:
            newX[index] = x.get(i)
        else:
            newX[index] = 0
    return newX           


In [51]:
#question 15
def calculate_w (yTrain,xTrain):
    yTrain = yTrain.copy()
    xTrain = xTrain.copy()
    m = svm_train(yTrain,xTrain,'-c 10 -t 0')
    w = np.zeros(36)
    for i in range(m.l):
        sv_indice = m.sv_indices[i]-1
        w = w + m.sv_coef[0][i]*sparse_to_dense(xTrain[sv_indice])
        
    print('||w|| = '+str(np.linalg.norm(w)))
    return(w)
    
    
        
yTrain,xTrain = read_data('satimage.scale')
newy3 = to_binary_class(yTrain,3)
w = calculate_w (newy3,xTrain)


[{2: 0.127273, 3: -0.0952381, 4: -0.289256, 5: 0.0322581, 6: 0.0545455, 7: -0.157895, 8: -0.265625, 9: -0.25, 10: -0.106796, 11: -0.242105, 12: -0.328125, 13: -0.0967742, 15: -0.434783, 16: -0.491525, 17: -0.125, 18: -0.0679612, 19: -0.542169, 20: -0.528455, 21: -0.107692, 22: 0.0873786, 23: -0.177778, 24: -0.344, 25: -0.15625, 26: -0.134615, 27: -0.517241, 28: -0.6, 29: -0.138462, 30: -0.188119, 31: -0.431579, 32: -0.546875, 33: -0.15625, 34: -0.126214, 35: -0.431579, 36: -0.484375}]
||w|| = 8.457084298367683


In [43]:
 #question 16
    
def kernel(xn,xm):
    xn = xn.copy()
    xm = xm.copy()
    return (1+np.dot(xn,xm))*(1+np.dot(xn,xm))
    
    
def calculate_Ein(y,x):
    y = y.copy()
    x = x.copy()
    m = svm_train(y,x,'-t 1 -d 2 -g 1 -r 1 -c 10')
    b = -m.rho[0]  
    E_in = 0
    for i in range(len(y)):
        gSVM = 0
        for j in range (m.l):
            SVM_index = m.sv_indices[j]-1
            gSVM = gSVM + (m.sv_coef[0][j] * kernel(sparse_to_dense(x[SVM_index]),sparse_to_dense(x[i])) )
        if (gSVM+b) * y[i] < 0:
            E_in += 1
    E_in = E_in/len(y)
    print ('number of SVM = ' + str(m.l))
    return E_in

In [44]:

newy1 = to_binary_class(yTrain,1)
print ('1 versus not 1 E_in: ' + str(calculate_Ein(newy1,xTrain)))
newy2 = to_binary_class(yTrain,2)
print ('2 versus not 2 E_in: ' + str(calculate_Ein(newy2,xTrain)))
newy3 = to_binary_class(yTrain,3)
print ('3 versus not 3 E_in: ' + str(calculate_Ein(newy3,xTrain)))
newy4 = to_binary_class(yTrain,4)
print ('4 versus not 4 E_in: ' + str(calculate_Ein(newy4,xTrain)))
newy5 = to_binary_class(yTrain,5)
print ('5 versus not 5 E_in: ' + str(calculate_Ein(newy5,xTrain)))


b = -0.0060891379351281335
b_cal = -0.006299331113029716
number of SVM = 145
1 versus not 1 E_in: 0.0006764374295377678
b = -2.5476250168295476
b_cal = -2.5474669002750874
number of SVM = 87
2 versus not 2 E_in: 0.0
b = -6.225146722120392
b_cal = -6.225151026620371
number of SVM = 433
3 versus not 3 E_in: 0.022322435174746337
b = -2.338586747086251
b_cal = -2.338350005023784
number of SVM = 712
4 versus not 4 E_in: 0.040135287485907556
b = 2.0362841350038057
b_cal = 2.0361439663783782
number of SVM = 259
5 versus not 5 E_in: 0.006764374295377677


In [70]:
#question 18
def Gkernel(xn, xm, gamma):
    return math.exp(-gamma * (np.linalg.norm(xn - xm)) ** 2)

def calculate_Eout(y,x,yTest,xTest,C,gamma):
    y = y.copy()
    x = x.copy()
    m = svm_train(y,x,'-s 0 -t 2 -c '+ str(C) + ' -g '+str(gamma) )
    b = -m.rho[0]
    '''
    while True:
        if abs(m.sv_coef[0][k]) != C:
            b_cal = m.sv_coef[0][k]/abs(m.sv_coef[0][k])
            for j in range(m.l):
                SVM_index = m.sv_indices[j]-1
                b_cal = b_cal - m.sv_coef[0][j]*Gkernel(sparse_to_dense(x[SVM_index]),sparse_to_dense(x[m.sv_indices[k]-1]),gamma)
            break
        else:
            k += 1
    print('b_cal'+ str(b_cal))
    '''
    E_out = 0
    for i in range(len(yTest)):
        gSVM = 0
        for j in range (m.l):
            SVM_index = m.sv_indices[j]-1
            gSVM = gSVM + (m.sv_coef[0][j] * Gkernel(sparse_to_dense(x[SVM_index]),sparse_to_dense(xTest[i]),gamma))
        if (gSVM + b) * yTest[i] < 0:
            E_out += 1
    E_out = E_out/len(yTest)
    return E_out


yTest, xTest = read_data('satimage.scale.t')
yTrain6 = to_binary_class(yTrain,6)

newYTest = to_binary_class(yTest,6)

In [47]:
print('C = 0.01, Eout = '+ str(calculate_Eout(yTrain6,xTrain,newYTest,xTest,0.01,10)))
print('C = 0.1, Eout = '+ str(calculate_Eout(yTrain6,xTrain,newYTest,xTest,0.1,10)))
print('C = 1, Eout = '+ str(calculate_Eout(yTrain6,xTrain,newYTest,xTest,1,10)))
print('C = 10, Eout = '+ str(calculate_Eout(yTrain6,xTrain,newYTest,xTest,10,10)))
print('C = 100, Eout = '+ str(calculate_Eout(yTrain6,xTrain,newYTest,xTest,100,10)))

number of SVM = 3843
C = 0.01, Eout = 0.235
number of SVM = 3883
C = 0.1, Eout = 0.1635
number of SVM = 3690
C = 1, Eout = 0.1065
number of SVM = 3703
C = 10, Eout = 0.097
number of SVM = 3703
C = 100, Eout = 0.097


In [48]:
#question 19
print('gamma = 0.1, Eout = ' + str(calculate_Eout(yTrain6,xTrain,newYTest,xTest,0.1,0.1)))
print('gamma = 1, Eout = ' + str(calculate_Eout(yTrain6,xTrain,newYTest,xTest,0.1,1)))
print('gamma = 10, Eout = '  + str(calculate_Eout(yTrain6,xTrain,newYTest,xTest,0.1,10)))
print('gamma = 100, Eout = ' + str(calculate_Eout(yTrain6,xTrain,newYTest,xTest,0.1,100)))
print('gamma = 1000, Eout = '+ str(calculate_Eout(yTrain6,xTrain,newYTest,xTest,0.1,1000)))

number of SVM = 1102
gamma = 0.1, Eout = 0.0985
number of SVM = 1175
gamma = 1, Eout = 0.07
number of SVM = 3883
gamma = 10, Eout = 0.1635
number of SVM = 4435
gamma = 100, Eout = 0.235
number of SVM = 4435
gamma = 1000, Eout = 0.235


In [ ]:
#question 20
def shuffle_xy(yTrain,xTrain,seed):
    yTrain = yTrain.copy()
    xTrain = xTrain.copy()
    random.Random(seed).shuffle(xTrain)
    random.Random(seed).shuffle(yTrain)
    return yTrain, xTrain
def best_gamma(index):
    y = yTrain6.copy()
    x = xTrain.copy()
    #Gamma: A = 0.1 B = 1 C = 10 D = 100 E = 1000 
    numOfGamma = [0,0,0,0,0]
    #print('number i = ' + str(index), end ='')
    shuffleYTrain ,shuffleXTrain = shuffle_xy(y,x,index)
    yV = shuffleYTrain[:200].copy()
    xV = shuffleXTrain[:200].copy()
    yT = shuffleYTrain[200:].copy()
    xT = shuffleXTrain[200:].copy()
    Eval = [0,0,0,0,0]
    Eval[0] = calculate_Eout(yT,xT,yV,xV,0.1,0.1)
    #print('EvalA ' + str(Eval[0]))
    Eval[1] = calculate_Eout(yT,xT,yV,xV,0.1,1)
    #print('EvalB ' + str(Eval[1]))
    Eval[2] = calculate_Eout(yT,xT,yV,xV,0.1,10)
    #print('EvalC ' + str(Eval[2]))
    Eval[3] = calculate_Eout(yT,xT,yV,xV,0.1,100)
    #print('EvalD ' + str(Eval[3]))
    Eval[4] = calculate_Eout(yT,xT,yV,xV,0.1,1000)
    #print('EvalE ' + str(Eval[4]))
    bestGamma = Eval.index(min(Eval))
    #print(' Gamma = ' + str(bestGamma))
    return bestGamma
    '''
    if index == 0:
        #print(' gamma = 0.1 won')
        return 0
        numOfGamma[0] += 1
    elif index == 1:
        #print(' gamma = 1 won')
        return 1
        numOfGamma[1] += 1
    elif index == 2:
        #print(' gamma = 10 won')
        return 2
        numOfGamma[2] += 1
    elif index == 3:
        #print(' gamma = 100 won')
        return 3
        numOfGamma[3] += 1
    elif index == 4:
        #print(' gamma = 1000 won')
        return 4
        numOfGamma[4] +=1
    #return numOfGamma
    '''
    


if __name__=='__main__':
    pool = mp.Pool(mp.cpu_count())
    gammaNum = pool.map(best_gamma,range(1000))
    #print(gammaNum)
    result = Counter(gammaNum)
    print('gamma = 0.1, number = ' + str(result[0]))
    print('gamma = 1, number = ' + str(result[1]))
    print('gamma = 10, number = ' + str(result[2]))
    print('gamma = 100, number = ' + str(result[3]))
    print('gamma = 1000, number = ' + str(result[4]))
    
    
    
